### **Tecnológico de Monterrey**

#### **Maestría en Inteligencia Artificial Aplicada**
#### **Clase**: Operaciones de Aprendizaje Automático
#### **Docentes**: Dr. Gerardo Rodríguez Hernández | Mtro. Ricardo Valdez Hernández | Mtro. Carlos Alberto Vences Sánchez

##### **Actividad**: Proyecto: Avance (Fase 1)
##### **Equipo 25**:
| Nombre | Matrícula |
|--------|-----------|
| Rafael Becerra García | A01796211 |
| Andrea Xcaret Gómez Alfaro | A01796384 |
| David Hernández Castellanos | A01795964 |
| Juan Pablo López Sánchez | A01313663 |
| Osiris Xcaret Saavedra Solís | A01795992 |

### Objetivos:

**Analisis de Requerimientos**
**Tarea**: Analiza la problemática a resolver siguiendo la liga con la descripción del dataset asignado.

**Manipulación y preparación de datos**
**Tarea**: Realizar tareas de Exploratory Data Analysis (EDA)  y limpieza de datos utilizando herramientas y bibliotecas específicas (Python, Pandas, DVC, Scikitlearn, etc.)

**Exploración y preprocesamiento de datos**
**Tarea**: Explorar y preprocesar los datos para identificar patrones, tendencias y relaciones significativas.

**Versionado de datos**
**Tarea**: Aplicar técnicas de versionado de datos para asegurar reproducibilidad y trazabilidad.

**Construcción, ajuste y evaluación de Modelos de Machine Learning**
**Tarea**: Construir, ajustar y evaluar modelos de Machine Learning utilizando técnicas y algoritmos apropiados al problema.

In [ ]:
# --- Importaciones e inicializaciones --- #

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual
# sns.set(style='whitegrid')
# %matplotlib inline

In [ ]:
# --- Cargar Dataset --- #

df = pd.read_csv('../data/processed/a01313663/obesity_estimation.csv')
print('Dataset de trabajo (df)', df.shape)

### Exploración inicial
Revisar información general, tipos de datos, primeros registros y estadísticas descriptivas.

In [ ]:
# --- Revisión inicial --- #

df.head()

In [ ]:
# Información general y tipos
df.info()

### Correcciones
Removemos columnas sin valor, corregimos datos en columnas, removemos valores atípicos obvios, imputamos valores faltantes.

In [ ]:
# --- Remover columna mixed_type_col ---#

# Justificación:
# - No tiene valores uniformes
# - No parece guardar información que sea valiosa

df.drop(columns=['mixed_type_col'], axis=1, inplace=True)

# Confirmamos que la columna fue removida
df.head()

In [ ]:
# --- Corrección de tipos de datos --- #

# Justificación:
# Todas las columnas son identificadas como objeto, por lo cual es necesario revisar y corregir
# los tipos de datos, para obtener datos adecuados en cada columna y poder actuar en ellos.

# Según la página base del dataset, tenemos valores:
# - Categóricos: Gender, CAEC, CALC, MTRANS, NObeyesdad
# - Enteros: FCVC, TUE
# - Flotantes: Age, Height, Weight, NCP, CH2O, FAF
# - Binarios: family_history_with_overweight, FAVC, SMOKE, SCC

# Sin embargo, de acuerdo a la exploración visual encontramos valores flotantes en FCVC y TUE, así que los consideraremos flotantes.
numeric_cols = ['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']

# Inspeccionemos los primeros 20 valores de cada columna numérica
print("Valores en columnas numéricas:\n")
for col in numeric_cols:
    print(f"{col}: {df[col].unique()[:20]}")

# Columnas binarias (yes, no)
binary_cols = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']

# Inspeccionemos los primeros 20 valores de cada columna binaria
print("\nValores en columnas binarias:\n")
for col in binary_cols:
    print(f"{col}: {df[col].unique()[:20]}")

# Columnas de tipo texto
object_cols = ['Gender', 'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']

# Inspeccionemos los primeros 20 valores de cada columna de texto
print("\nValores en columnas de texto:\n")
for col in object_cols:
    print(f"{col}: {df[col].unique()[:20]}")


In [ ]:
# --- Corrección y limpieza --- #

# Justificación: encontramos algunas cosas irregulares, por ejemplo:
# - Espacios en blanco alreadedor del valor numérico: ' 3.0 '
# - Valores textuales: 'invalid' o ' NAN ' o '?'
# - Encontramos diferentes cadenas con diferente construcción de mayúsculas y minúsculas

# Convertir todas las columnas numéricas a float
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convertimos las columnas que identificamos como binarias a valores binarios (0 y 1)
binary_map = {'yes': 1, 'no': 0}
for col in binary_cols:
    df[col] = (
        df[col]
        .astype(str)              # Convertir todo a string
        .str.strip()              # Eliminar espacios
        .str.lower()              # Uniformar a minúsculas
        .replace({'nan': np.nan}) # Convertir texto "nan"/" NAN " a NaN real
        .map(binary_map)          # Mapear yes/no a 1/0
    )

# Convertir a enteros con soporte para NaN
df[binary_cols] = df[binary_cols].astype('Int64')

# Aplicamos strip y lower en todas las columnas de texto
for col in object_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()

# Reemplazar valores no numéricos por NaN
df.replace({'nan': np.nan, '?': np.nan, 'error': np.nan, 'invalid': np.nan, 'n/a': np.nan, 'null': np.nan}, inplace=True)

# Verificar resultados
print(df.info())
df.head()

In [ ]:
# --- Validación visual ---#

# Reimprimimos los valores de muestra una vez corregidos
print("Valores en columnas numéricas:\n")
for col in numeric_cols:
    print(f"{col}: {df[col].unique()[:20]}")

# Inspeccionemos los primeros 20 valores de cada columna binaria
print("\nValores en columnas binarias:\n")
for col in binary_cols:
    print(f"{col}: {df[col].unique()[:20]}")

# Inspeccionemos los primeros 20 valores de cada columna de texto
print("\nValores en columnas de texto:\n")
for col in object_cols:
    print(f"{col}: {df[col].unique()[:20]}")

In [ ]:
# --- Filtrado de outliers obvios --- #

# Justificación:
# - Hay valores muy fuera de los esperados o simplemente inválidos

# Definir rangos normales para cada columna numérica
valid_ranges = {
    'Age': (5, 120),         # años
    'Height': (1.3, 2.2),    # metros
    'Weight': (30, 200),     # kg
    'FCVC': (1, 3),          # frecuencia comida principal
    'NCP': (1, 3),           # número de comidas
    'CH2O': (1, 3),          # litros de agua
    'FAF': (0, 5),           # actividad física
    'TUE': (0, 3)            # tiempo frente a pantalla
}

# Aplicar filtros: si el valor cae fuera del rango establecido como normal, poner NaN
for col, (min_val, max_val) in valid_ranges.items():
    df.loc[(df[col] < min_val) | (df[col] > max_val), col] = np.nan

# Verificar resultados
for col in valid_ranges.keys():
    print(f"{col}: valores únicos (primeros 20) después de filtrar outliers")
    print(df[col].unique()[:20])
    print("----")

In [ ]:
# --- Imputación de NaN y limpieza final --- #

# Columnas numéricas: imputar con mediana
for col in numeric_cols:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)
    print(f"Columna {col}: mediana imputada = {median_value}")

# Columnas categóricas: imputar con moda
for col in object_cols:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)
    print(f"Columna {col}: modo imputado = '{mode_value}'")

# Columnas binarias: imputar con moda
for col in binary_cols:
    moda = df[col].mode(dropna=True)[0]
    df[col] = df[col].fillna(moda)

# Buscar registros duplicados, es decir, que sean exactamente iguales
dups = df.duplicated().sum()
print(f'Registros duplicados: {dups}\n')

# En caso de encontrar alguno, eliminarlos
if dups > 0:
    print('Eliminando duplicados ...')
    df = df.drop_duplicates()
    dups = df.duplicated().sum()
    print(f'Registros duplicados: {dups}\n')

# Veamos las nuevas dimensiones del dataset
print('Nuevas dimensiones del dataset (df)', df.shape)

# Verificación final
print("\nInformación final del dataset:")
print(df.info())
df.head()

In [ ]:
# --- Guardar versión limpia después del procesamiento realizado --- #

# Generamos una copia del dataframe (df_clean) para continuar en ella el EDA
df_clean = df.copy()

# Y versionamos nuestra copia limpia
df_clean.to_csv('../data/processed/a01313663/obesity_estimation_clean.csv', index=False)

### Inspección Visual
Revisemos las estadísticas por tipo de columna y el conteo de valores nulos por columna.

In [ ]:
# --- Estadísticas descriptivas --- #

# Columnas numéricas
display(df_clean[numeric_cols].describe())

# Columnas binarias (estadísticas tipo object)
display(df_clean[binary_cols].astype('object').describe())

# Columnas de texto
display(df_clean[object_cols].describe())

In [ ]:
# Conteo de nulos por columna
df_clean.isnull().sum()

### 5️⃣ Identificación de valores inconsistentes o outliers
- Se pueden agregar funciones o visualizaciones específicas para detectar valores fuera de rango o inconsistentes.

In [ ]:
# ----------------------------===
# 4️⃣ Visualización: EDA numérico completo
# ----------------------------===

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Columnas numéricas
numeric_cols = df.select_dtypes(include=np.number).columns

# 1️⃣ Boxplots por columna
for col in numeric_cols:
    plt.figure(figsize=(8,3))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot de {col}')
    plt.show()

# 2️⃣ Histogramas con KDE
for col in numeric_cols:
    plt.figure(figsize=(8,3))
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(f'Histograma de {col} con KDE')
    plt.show()

# 3️⃣ Matriz de correlación
plt.figure(figsize=(10,8))
corr_matrix = df[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de correlación de variables numéricas")
plt.show()


In [ ]:
# ----------------------------===
# 5️⃣ EDA para columnas categóricas
# ----------------------------===

# Columnas categóricas
cat_cols = df.select_dtypes(include='object').columns

for col in cat_cols:
    plt.figure(figsize=(8,4))
    
    # Conteo y proporción
    counts = df[col].value_counts()
    percents = df[col].value_counts(normalize=True) * 100
    
    # Mostrar en consola
    print(f"\nColumna: {col}")
    print("Conteo de valores:")
    print(counts)
    print("Porcentaje de cada categoría:")
    print(percents.round(2))
    
    # Gráfico de barras
    sns.barplot(x=counts.index, y=counts.values)
    plt.title(f'Conteo de categorías en {col}')
    plt.ylabel('Cantidad de registros')
    plt.xlabel(col)
    plt.xticks(rotation=45)
    plt.show()


### 6️⃣ Checklist de limpieza
- Eliminar o imputar valores nulos
- Corregir inconsistencias
- Eliminar duplicados
- Tratar outliers según criterio
- Guardar la versión limpia y versionarla con DVC

In [ ]:
# Ejemplo: guardar versión limpia (después de aplicar limpieza)
# df_clean = df.copy()  # aplicar limpieza aquí
# df_clean.to_csv('data/processed/a01313663/obesity_estimation_clean.csv', index=False)

In [ ]:
# ----------------------------===
# 6️⃣ Resumen completo de EDA
# ----------------------------===

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ---------------------------------
# 1️⃣ Estadísticas descriptivas
# ---------------------------------
print("----= Estadísticas descriptivas (numéricas) ----=")
display(df.describe())

print("\n----= Conteo de valores (categóricas) ----=")
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    counts = df[col].value_counts()
    percents = df[col].value_counts(normalize=True) * 100
    print(f"\nColumna: {col}")
    print("Conteo de valores:")
    print(counts)
    print("Porcentaje de cada categoría:")
    print(percents.round(2))

# ---------------------------------
# 2️⃣ Boxplots + Histogramas numéricos
# ---------------------------------
numeric_cols = df.select_dtypes(include=np.number).columns

for col in numeric_cols:
    # Boxplot
    plt.figure(figsize=(8,3))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot de {col}')
    plt.show()
    
    # Histograma + KDE
    plt.figure(figsize=(8,3))
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(f'Histograma de {col} con KDE')
    plt.show()

# ---------------------------------
# 3️⃣ Correlación
# ---------------------------------
plt.figure(figsize=(10,8))
corr_matrix = df[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de correlación de variables numéricas")
plt.show()

# ---------------------------------
# 4️⃣ Conteo visual de columnas categóricas
# ---------------------------------
for col in cat_cols:
    plt.figure(figsize=(8,4))
    counts = df[col].value_counts()
    sns.barplot(x=counts.index, y=counts.values)
    plt.title(f'Conteo de categorías en {col}')
    plt.ylabel('Cantidad de registros')
    plt.xlabel(col)
    plt.xticks(rotation=45)
    plt.show()
